# AI-Based Hybrid Text Analysis: Topic Modeling & Network Analysis
## BERTopic, SNA, and LLM Prompting (Azure OpenAI)

이 노트북은 연구 보고서에서 제안한 **'AI 기반 하이브리드 텍스트 분석 프레임워크'**를 구현합니다.

**분석 프로세스:**
1. **Data Loading & Preprocessing:** 뉴스 데이터 로드 및 전처리
2. **BERTopic Analysis:** 동적 토픽 모델링을 통한 주요 이슈 및 시계열 변화 추적
3. **Semantic Network Analysis (SNA):** 키워드 동시 출현 네트워크 시각화 (기술 생태계 지도)
4. **LLM Prompt Generation:** 분석 결과를 바탕으로 심층 해석을 위한 LLM 프롬프트 자동 생성
5. **LLM Integration (Azure):** Azure OpenAI를 연동하여 실제 분석 리포트 자동 생성

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from collections import Counter
from itertools import combinations
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Azure OpenAI 및 환경변수 로드
from openai import AzureOpenAI
from dotenv import load_dotenv

# 시각화 설정
sns.set(style="whitegrid")
font_name = "NanumGothic"
plt.rcParams["font.family"] = font_name
plt.rcParams["axes.unicode_minus"] = False

# 데이터 경로
DATA_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

# .env 파일 로드 (프로젝트 루트 경로 기준)
load_dotenv(dotenv_path="../.env")

print(f"Libraries Loaded & Environment Variables Checked. Using font: {font_name}")

Libraries Loaded & Environment Variables Checked. Using font: NanumGothic


In [2]:
# 1. Data Loading & Preprocessing
def load_and_preprocess():
    try:
        sam_df = pd.read_csv(os.path.join(DATA_DIR, "samsung_news.csv"))
        sk_df = pd.read_csv(os.path.join(DATA_DIR, "skhynix_news.csv"))
    except FileNotFoundError:
        print("데이터 파일이 없습니다. 더미 데이터를 생성합니다.")
        sam_df = pd.DataFrame({'date': ['2024-01-01'], 'title': ['삼성전자 GAA 파운드리'], 'content': ['삼성전자가 3나노 GAA 공정 양산을 시작했다.']})
        sk_df = pd.DataFrame({'date': ['2024-01-01'], 'title': ['SK하이닉스 HBM3E'], 'content': ['SK하이닉스가 HBM3E를 엔비디아에 독점 공급한다.']})
    
    sam_df['company'] = 'Samsung'
    sk_df['company'] = 'SKHynix'
    df = pd.concat([sam_df, sk_df], ignore_index=True)
    df['text'] = df['title'].fillna('') + " " + df['content'].fillna('')
    df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
    df = df.dropna(subset=['date']).sort_values('date')
    
    def clean_text(text):
        text = str(text)
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    df['processed_text'] = df['text'].apply(clean_text)
    return df

df = load_and_preprocess()
print(f"Total Data: {len(df)} records")

Total Data: 2956 records


In [3]:
# 2. BERTopic Analysis
def run_bertopic(docs, min_topic_size=30):
    print("Initializing BERTopic...")
    korean_stopwords = ['기자', '배포', '금지', '무단', '전재', '위해', '통해', '관련', '대한', '이번', '있다', '했다', '밝혔다']
    vectorizer_model = CountVectorizer(stop_words=korean_stopwords)

    topic_model = BERTopic(
        embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
        vectorizer_model=vectorizer_model,
        min_topic_size=min_topic_size,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(docs)
    return topic_model, topics

sample_df = df.tail(1000).copy()
docs = sample_df['processed_text'].tolist()

try:
    topic_model, topics = run_bertopic(docs)
    print("BERTopic Training Complete.")
    freq = topic_model.get_topic_info(); 
    print(freq.head(5))
except Exception as e:
    print(f"BERTopic Error: {e}")

2025-11-23 23:10:28,502 - BERTopic - Embedding - Transforming documents to embeddings.


Initializing BERTopic...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2025-11-23 23:10:36,496 - BERTopic - Embedding - Completed ✓
2025-11-23 23:10:36,497 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-23 23:10:48,464 - BERTopic - Dimensionality - Completed ✓
2025-11-23 23:10:48,465 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-23 23:10:48,498 - BERTopic - Cluster - Completed ✓
2025-11-23 23:10:48,501 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-23 23:10:49,449 - BERTopic - Representation - Completed ✓


BERTopic Training Complete.
   Topic  Count                     Name  \
0     -1     13  -1_삼성은_희망디딤돌_삼성희망디딤돌_삼성   
1      0    937       0_반도체_있는_ai_sk하이닉스   
2      1     50            1_제품_대비_매출_수요   

                                      Representation  \
0  [삼성은, 희망디딤돌, 삼성희망디딤돌, 삼성, 교육, 명장, 채용, 공채, csr,...   
1  [반도체, 있는, ai, sk하이닉스, 메모리, 기술, 함께, 다양한, 삼성전자, 위한]   
2     [제품, 대비, 매출, 수요, 판매, 실적, 전분기, 프리미엄, 시장, 삼성전자는]   

                                 Representative_Docs  
0  [청년과 함께하는 행복한 동행, 삼성희망디딤돌 대전센터 개소식 초여름의 푸릇한 기운...  
1  [[SK하이닉스 41주년] “빛나는 40+1” 40년 기술력 다져 No.1으로 우뚝...  
2  [삼성전자, 2023년 4분기 실적 발표 삼성전자는 연결 기준으로 매출 67.78조...  


In [4]:
# 3. Semantic Network Analysis (SNA)
def extract_keywords_regex(text):
    if not isinstance(text, str): return []
    tech_terms = re.findall(r'\b[A-Za-z][A-Za-z0-9]+\b', text)
    korean_nouns = re.findall(r'[가-힣]{2,}', text)
    target_keywords = set(['HBM', 'DDR', 'NAND', 'AI', '반도체', '파운드리', '수율', '투자', '양산', 
                          '엔비디아', 'TSMC', 'GAA', '이익', '적자', 'CXL', 'PIM', 'SK하이닉스', '삼성전자'])
    found = []
    for word in tech_terms + korean_nouns:
        w_upper = word.upper()
        for target in target_keywords:
            if target in w_upper or w_upper in target:
                found.append(target)
                break
    return list(set(found))

def draw_semantic_network(df, company_name, threshold=5):
    subset = df[df['company'] == company_name].copy()
    subset['keywords'] = subset['processed_text'].apply(extract_keywords_regex)
    edge_list = []
    for keywords in subset['keywords']:
        if len(keywords) > 1:
            edge_list.extend(combinations(sorted(keywords), 2))
    edge_counts = Counter(edge_list)
    G = nx.Graph()
    for (u, v), count in edge_counts.items():
        if count >= threshold: G.add_edge(u, v, weight=count)
    return G

G_sk = draw_semantic_network(sample_df, "SKHynix", threshold=20)
G_sam = draw_semantic_network(sample_df, "Samsung", threshold=20)
print("Networks Built.")

Networks Built.


In [5]:
# 4. LLM Prompt Generation
def generate_llm_prompt(G, company_name):
    degree_dict = dict(G.degree(weight='weight'))
    top_nodes = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)[:5]
    top_keywords = [k[0] for k in top_nodes]
    
    prompt = f"""
    [역할]
    당신은 반도체 산업 전문 애널리스트입니다. 아래의 데이터 분석 결과를 바탕으로 통찰력 있는 보고서를 작성해 주세요.
    
    [분석 데이터: {company_name}]
    1. 핵심 연결 키워드 (Top 5): {top_keywords}
    2. 네트워크 특징: '{top_keywords[0]}' 키워드가 중심 허브 역할을 하며 기술 생태계를 주도함.
    
    [요청 사항]
    1. **전략 진단:** 현재 {company_name}의 기술 전략이 '집중형'인지 '분산형'인지 진단해 주세요.
    2. **성과 예측:** 특히 '{top_keywords[0]}' 기술이 AI 산업 트렌드와 결합하여 향후 어떤 재무적 성과를 낼지 추론해 주세요.
    3. **약한 신호(Weak Signal):** 네트워크 주변부에 위치했을 가능성이 있는 차세대 기술을 제안하고 그 이유를 설명해 주세요.
    """
    return prompt

target_prompt = generate_llm_prompt(G_sk, "SK하이닉스")
print(target_prompt)


    [역할]
    당신은 반도체 산업 전문 애널리스트입니다. 아래의 데이터 분석 결과를 바탕으로 통찰력 있는 보고서를 작성해 주세요.

    [분석 데이터: SK하이닉스]
    1. 핵심 연결 키워드 (Top 5): ['반도체', 'SK하이닉스', 'AI', 'HBM', '양산']
    2. 네트워크 특징: '반도체' 키워드가 중심 허브 역할을 하며 기술 생태계를 주도함.

    [요청 사항]
    1. **전략 진단:** 현재 SK하이닉스의 기술 전략이 '집중형'인지 '분산형'인지 진단해 주세요.
    2. **성과 예측:** 특히 '반도체' 기술이 AI 산업 트렌드와 결합하여 향후 어떤 재무적 성과를 낼지 추론해 주세요.
    3. **약한 신호(Weak Signal):** 네트워크 주변부에 위치했을 가능성이 있는 차세대 기술을 제안하고 그 이유를 설명해 주세요.
    


In [6]:
# 5. Azure OpenAI Integration

# 환경 변수 확인 (디버깅용, 실제 키 값은 출력하지 않음)
print("Azure OpenAI Config Check:")
print(f"- Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"- Deployment Name: {os.getenv('AZURE_OPENAI_LLM_DEPLOYMENT')}")
print(f"- API Version: {os.getenv('AZURE_OPENAI_API_VERSION')}")

def query_azure_llm(prompt):
    # 클라이언트 초기화
    client = AzureOpenAI(
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    )
    
    deployment_name = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT") # e.g., 'gpt-4o'
    
    try:
        print("🤖 Azure LLM에게 분석 요청 중... (잠시만 기다려주세요)")
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled in semiconductor industry analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Error calling Azure OpenAI: {e}"

# 실행 예시 (환경 변수가 올바르게 설정되어 있다면 실행됨)
if os.getenv('AZURE_OPENAI_API_KEY'):
    analysis_result = query_azure_llm(target_prompt)
    
    print("\n" + "="*20 + " Azure LLM 분석 리포트 " + "="*20 + "\n")
    print(analysis_result)
else:
    print("\n⚠️ .env 파일에 AZURE_OPENAI_API_KEY가 설정되지 않았습니다.")

Azure OpenAI Config Check:
- Endpoint: https://hspar-m7k2pfor-swedencentral.openai.azure.com/
- Deployment Name: gpt-4o
- API Version: 2024-12-01-preview
🤖 Azure LLM에게 분석 요청 중... (잠시만 기다려주세요)

==================== Azure LLM 분석 리포트 ====================

### SK하이닉스의 기술 전략 및 미래 전망 보고서

#### 1. **전략 진단: 집중형 전략**
SK하이닉스의 핵심 연결 키워드와 네트워크 특징을 분석한 결과, '반도체' 키워드가 중심 허브 역할을 하며 기술 생태계를 주도하고 있습니다. 이는 SK하이닉스가 특정 핵심 기술에 집중하여 가치를 극대화하는 **집중형 전략**을 채택하고 있음을 보여줍니다.

특히, 'HBM(High Bandwidth Memory)'과 'AI'와의 연결성은 SK하이닉스가 메모리 반도체의 고성능·고효율화에 역량을 집중하고 있음을 시사합니다. HBM은 AI와 고성능 컴퓨팅(HPC) 분야에서 필수적인 기술로 떠오르고 있으며, SK하이닉스는 이러한 미래 성장 동력 분야에 강력히 투자하고 있는 것으로 보입니다. 이 전략은 경쟁사와 차별화된 기술 리더십 확보와 시장 점유율 확대를 목표로 하고 있습니다.

---

#### 2. **성과 예측: 반도체 기술과 AI 산업의 결합**
AI 산업의 급격한 성장과 함께 반도체, 특히 메모리 반도체의 중요성이 더욱 부각되고 있습니다. AI 시스템은 방대한 데이터를 처리하기 위해 고속·고성능 메모리 기술을 필요로 하며, HBM은 이를 충족시키는 핵심 기술로 자리하고 있습니다. SK하이닉스는 HBM3을 포함한 차세대 메모리 제품을 양산 중이며, 이를 통해 AI와 HPC 분야에서 폭발적인 수요 증가를 예상할 수 있습니다.

**재무적 성과 전망**:
- **매출 증가**: AI 산업의 확장은 반도체 수요를 견인하